In [18]:
import pypsa
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import plotly.graph_objects as go

In [19]:
print(random.randint(0, 10))

2


In [20]:
nodes = pd.read_csv('DATA - Substation Coordinates.csv')
edges = pd.read_csv('DATA - TRANSMISSION_LINES.csv', usecols=['OBJECTID', 'NAME', 'CAPACITY_KV', 'SHAPE_Length', 'OPERATIONAL_STATUS'])
nodes.drop(['OPERATIONAL_STATUS', 'ATTRIBUTE_SOURCE',
            'CUSTODIAN_AGENCY', 'CUSTODIAN_LICENSING'], axis=1)

# spare_edges = edges[edges['NAME'].str.contains(' to ')]

edges = edges[edges.OPERATIONAL_STATUS == 'Operational']
spare_edges = edges[~edges.NAME.str.contains(' to ')]
edges = edges[edges.NAME.str.contains(' to ')]

In [21]:
edges[['f', 't']] = edges.NAME.str.split(' to ', expand=True)

In [22]:
def get_connected_x_y(bus, edges, nodes):
    if bus in edges.t.to_list():
        for i in edges[edges.f==bus].f:
            if i in nodes.NAME.to_list():
                x = nodes[nodes.NAME==i].LATITUDE
                y = nodes[nodes.NAME==i].LATITUDE - edges[edges.f].SHAPE_Length
                return x, y
    if bus in edges.f.to_list():
        for i in edges[edges.t==bus].t:
            if i in nodes.NAME.to_list():
                x = nodes[nodes.NAME==i].LATITUDE
                y = nodes[nodes.NAME==i].LATITUDE - edges[edges.f].SHAPE_Length
                return x, y
    return 145, -30

In [23]:
nodes_ls = []
buses = np.unique(list(set(edges.f.to_list() + edges.t.to_list())))
edges.drop_duplicates(['NAME'],inplace=True)
for i in buses:
    if i in nodes.NAME.to_list():
        y = nodes[nodes.NAME==i].LATITUDE.to_list()[0]
        x = nodes[nodes.NAME==i].LONGITUDE.to_list()[0]
    else:
        x, y = get_connected_x_y(i, edges, nodes)

    nodes_ls.append((i, x, y))

In [24]:
df1 = pd.DataFrame(nodes_ls, columns=['NAME', 'LONGITUDE', 'LATITUDE'])
df1.head(100)

,NAME,LONGITUDE,LATITUDE
0,910 Tee,145.000000,-30.000000
1,911 Tee,145.000000,-30.000000
2,926 Tee,145.000000,-30.000000
3,927 Tee,145.000000,-30.000000
4,92A Tee,145.000000,-30.000000
...,...,...,...
95,Baulkham Hills,150.968721,-33.781827
96,Bayswater,150.946099,-32.399458
97,Bayswater Power Station,145.000000,-30.000000
98,Beaconsfield North,151.187904,-33.914333


In [25]:
network = pypsa.Network()
for i in buses:
    y = df1[df1.NAME==i].LATITUDE.to_list()[0]
    x = df1[df1.NAME==i].LONGITUDE.to_list()[0]
    network.add(
    "Bus",
    "{}".format(i),
    x=x, y=y
    )

for index, row in edges.iterrows():
    network.add(
        "Line",
        "{}".format(row["NAME"]),
        bus0=row["f"],
        bus1=row["t"],
        # p_nom=1,
    )
# network.iplot(layouter=nx.spring_layout)
# network.iplot(mapbox=True)
fig = go.Figure(network.iplot(mapbox=True))
fig.write_image("aus sample.png")

In [26]:
G = network.graph()
pos = dict()
# pos = nx.spring_layout(G)
for x, i in df1.iterrows():
    if i.LATITUDE != -30.0:
        pos[i.NAME] = [i.LONGITUDE, i.LATITUDE]
    # else:
    #     pos[i.NAME] = None
# pos = nx.spring_layout(G, pos=pos, center=[145,-30])
pos = nx.kamada_kawai_layout(G, pos=pos, center=[145,-30], dim=2, scale=1, weight=None)
# pos = nx.spectral_layout(G, center= [145,-30], dim=2)
# pos = nx.random_layout(G, center= [145,-30], dim=2)
for x, i in df1.iterrows():
    if i.LATITUDE != -30.0:
        pos[i.NAME] = [i.LONGITUDE, i.LATITUDE]




KeyError: '910 Tee'

In [ ]:
nx.draw(G, pos=pos)

In [ ]:
for i in pos.keys():
    # print(i)
    network.buses.loc[i, 'x'] = pos[i][0]
    network.buses.loc[i, 'y'] = pos[i][1]
    # network.buses[network.buses == i].y = pos[i][1]
network.iplot(geomap=True)